## Imports and setting up db connection

In [1]:
import yfinance as yf
import pandas as pd
import sqlite3

from data import Data

data = Data()

con = data.con

symbols = ["SPY", "IEF", "TLT", "GLD", "VNQ"]

## Update db data

In [2]:
data.updateEodData()

[*********************100%***********************]  1 of 1 completed
SPY is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
IEF is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
TLT is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
GLD is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
VNQ is already up to date (latest date: 2020-06-12)


## Reading data from the db

In [3]:
data.getEodData("TLT", start="2019-04-03", end="2020-01-01")

,open,high,low,close,adj_close,volume
date,,,,,,
2019-04-03,123.61,123.87,123.37,123.52,120.6112,10407000
2019-04-04,123.60,123.89,123.44,123.87,120.9530,7503400
2019-04-05,123.62,124.17,123.58,124.03,121.1092,6321800
2019-04-08,123.83,123.95,123.52,123.59,120.6796,4446400
2019-04-09,124.22,124.34,123.77,123.98,121.0604,7020300
...,...,...,...,...,...,...
2019-12-24,135.92,137.06,135.89,136.84,135.8637,4667400
2019-12-26,136.91,137.22,136.51,137.17,136.1913,5290200
2019-12-27,137.44,137.63,137.29,137.32,136.3403,7300500
